# Project: Extract, Transfrom and Load GDP Data


## Introduction

We will create a complete ETL pipeline for accessing data from a website and processing it to meet the requirements.


## Project Scenario:

An international firm that is looking to expand its business in different countries across the world has recruited you. You have been hired as a junior Data Engineer and are tasked with creating an automated script that can extract the list of all countries in order of their GDPs in billion USDs (rounded to 2 decimal places), as logged by the International Monetary Fund (IMF). Since IMF releases this evaluation twice a year, this code will be used by the organization to extract the information as it is updated.

The required data seems to be available on the URL mentioned below:

URL:
https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29

## Objectives

 - Write a data extraction function to retrieve the relevant information from the required URL.
 - Transform the available GDP information into 'Billion USD' from 'Million USD'.
 - Load the transformed information to the required CSV file and as a database file.
 - Run the required query on the database.
 - Log the progress of the code with appropriate timestamps.


---

## Setup


The libraries needed for the code are as follows:

* requests - The library used for accessing the information from the URL.

* bs4 - The library containing the BeautifulSoup function used for webscraping.

* pandas - The library used for processing the extracted data, storing it to required formats and communicating with the databases.

* sqlite3 - The library required to create a database server connection.

* numpy - The library required for the mathematical rounding operation as required in the objectives.

* datetime - The library containing the function datetime used for extracting the timestamp for logging purposes.

### Importing Required Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

### Setting Path Configuration

In [2]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

### Extract

In [3]:
def extract(url,table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
      col = row.find_all('td') 
      if len(col)!=0:
        if col[0].find('a') is not None and '—' not in col[2]:
          data_dict = {"Country": col[0].a.contents[0],"GDP_USD_millions": col[2].contents[0]}
          temp_df = pd.DataFrame(data_dict, index=[0])
          df = pd.concat([df,temp_df], ignore_index=True)
    return df

### Transform

In [4]:
def transform(df):
    GDP_list = df["GDP_USD_millions"].tolist()
    GDP_list = [float("".join(x.split(','))) for x in GDP_list]
    GDP_list = [np.round(x/1000,2) for x in GDP_list]
    df["GDP_USD_millions"] = GDP_list
    df=df.rename(columns = {"GDP_USD_millions":"GDP_USD_billions"})
    return df

### Load

In [5]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

In [6]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

### Query

In [7]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

### Log Progress

In [8]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./etl_project_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

### Function Call

In [9]:
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('World_Economies.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
          Country  GDP_USD_billions
0   United States          26854.60
1           China          19373.59
2           Japan           4409.74
3         Germany           4308.85
4           India           3736.88
..            ...               ...
64          Kenya            118.13
65         Angola            117.88
66           Oman            104.90
67      Guatemala            102.31
68       Bulgaria            100.64

[69 rows x 2 columns]
